In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


(1) CNN-LSTM full images

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import TimeDistributed, Reshape, LSTM
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import os
from google.colab import drive

# Set random seed for reproducibility
tf.keras.utils.set_random_seed(492)

# Load data from Google Drive
def import_data(base_path, class_names, img_height=224, img_width=224):
    images = []
    labels = []
    for class_idx, class_name in enumerate(class_names):
        class_folder = os.path.join(base_path, class_name)
        for img_name in os.listdir(class_folder):
            img_path = os.path.join(class_folder, img_name)
            try:
                img = load_img(img_path, target_size=(img_height, img_width))
                img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
                images.append(img_array)
                labels.append(class_idx)
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
    return np.array(images), np.array(labels)

# Define CNN model
def create_cnn_lstm_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Reshape((54, 54*64)),
        LSTM(64, return_sequences=False),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])

    # Compile with Adam optimizer
    model.compile(optimizer=Adam(learning_rate=0.001),
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

    return model

# Main
base_path = '/content/drive/My Drive/mbpose/plank'
class_names = ['Incorrect_low', 'Correct', 'Incorrect_high']
img_height, img_width = 224, 224

# Load data
x, y = import_data(base_path, class_names, img_height, img_width)
num_classes = len(np.unique(y))

# Create 6 stratified folds
skf = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)
folds = list(skf.split(x, y))
train_val_folds = folds[:5]
test_fold = folds[5]

precision_list, recall_list, accuracy_list, f1_list = [], [], [], []

# 5-Fold Cross-Validation
for fold_index, (train_index, val_index) in enumerate(train_val_folds, start=1):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    print(f"\n=== Fold {fold_index} ===")
    print("Class distribution:", pd.Series(y_val).value_counts().to_dict())

    # Class weights for imbalance
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
    class_weights = {i: class_weights[i] for i in range(len(class_weights))}

    # Create model
    model = create_cnn_lstm_model(input_shape=(img_height, img_width, 3), num_classes=num_classes)

    # Early stopping
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Train
    model.fit(
        x_train, y_train, epochs=30, batch_size=32, validation_data=(x_val, y_val),
        verbose=1, class_weight=class_weights, callbacks=[early_stop]
    )

    # Evaluate
    predictions = np.argmax(model.predict(x_val), axis=1)
    precision = precision_score(y_val, predictions, average='macro', zero_division=0)
    recall = recall_score(y_val, predictions, average='macro', zero_division=0)
    accuracy = accuracy_score(y_val, predictions)
    f1 = f1_score(y_val, predictions, average='macro', zero_division=0)

    print("Confusion Matrix:")
    print(confusion_matrix(y_val, predictions))
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")

    precision_list.append(precision)
    recall_list.append(recall)
    accuracy_list.append(accuracy)
    f1_list.append(f1)

# Report average scores
print("\n=== 5-Fold Cross-Validation Summary ===")
print(f"Avg Precision: {np.mean(precision_list):.4f}")
print(f"Avg Recall:    {np.mean(recall_list):.4f}")
print(f"Avg Accuracy:  {np.mean(accuracy_list):.4f}")
print(f"Avg F1 Score:  {np.mean(f1_list):.4f}")

# === Final Testing on Fold 6 ===
train_index = np.concatenate([f[0] for f in train_val_folds])
test_index = test_fold[1]
x_train, x_test = x[train_index], x[test_index]
y_train, y_test = y[train_index], y[test_index]

# Class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

# Train final model
final_model = create_cnn_lstm_model(input_shape=(img_height, img_width, 3), num_classes=num_classes)
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

final_model.fit(
    x_train, y_train, epochs=30, batch_size=32, validation_split=0.1,
    class_weight=class_weights, callbacks=[early_stop], verbose=1
)

# Final test evaluation
test_predictions = np.argmax(final_model.predict(x_test), axis=1)
test_accuracy = accuracy_score(y_test, test_predictions)
test_f1 = f1_score(y_test, test_predictions, average='macro', zero_division=0)
test_precision = precision_score(y_test, test_predictions, average='macro', zero_division=0)
test_recall = recall_score(y_test, test_predictions, average='macro', zero_division=0)

print("\n=== Final Test Fold (Fold 6) Evaluation ===")
print("Confusion Matrix:")
print(confusion_matrix(y_test, test_predictions))
print(f"Test Accuracy:  {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall:    {test_recall:.4f}")
print(f"Test F1 Score:  {test_f1:.4f}")


=== Fold 1 ===
Class distribution: {2: 58, 0: 57, 1: 53}


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.3172 - loss: 1.1275 - val_accuracy: 0.3393 - val_loss: 1.0983
Epoch 2/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 87s 3s/step - accuracy: 0.3750 - loss: 1.0903 - val_accuracy: 0.3750 - val_loss: 1.0748
Epoch 3/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 140s 3s/step - accuracy: 0.3730 - loss: 1.0871 - val_accuracy: 0.4464 - val_loss: 1.0533
Epoch 4/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 138s 3s/step - accuracy: 0.4678 - loss: 1.0107 - val_accuracy: 0.5536 - val_loss: 0.9413
Epoch 5/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - accuracy: 0.5928 - loss: 0.9256 - val_accuracy: 0.5655 - val_loss: 0.8761
Epoch 6/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 84s 3s/step - accuracy: 0.6232 - loss: 0.8438 - val_accuracy: 0.6012 - val_loss: 0.8759
Epoch 7/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 77s 3s/step - accuracy: 0.7152 - loss: 0.7155 - val_accuracy: 0.7024 - val_loss: 0.6417
Epoch 8/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 84s 3s/step - accuracy: 0.7594 - loss: 0.5762 - val_accuracy: 0.8452 - val_los

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 91s 3s/step - accuracy: 0.3318 - loss: 1.1321 - val_accuracy: 0.3274 - val_loss: 1.0999
Epoch 2/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 134s 3s/step - accuracy: 0.3061 - loss: 1.0988 - val_accuracy: 0.3393 - val_loss: 1.0980
Epoch 3/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 88s 3s/step - accuracy: 0.3462 - loss: 1.0943 - val_accuracy: 0.3810 - val_loss: 1.0847
Epoch 4/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 78s 3s/step - accuracy: 0.3908 - loss: 1.0845 - val_accuracy: 0.4226 - val_loss: 1.0160
Epoch 5/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - accuracy: 0.4582 - loss: 1.0335 - val_accuracy: 0.6190 - val_loss: 0.8998
Epoch 6/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.5476 - loss: 0.9802 - val_accuracy: 0.6488 - val_loss: 0.8369
Epoch 7/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 77s 3s/step - accuracy: 0.6154 - loss: 0.8704 - val_accuracy: 0.7500 - val_loss: 0.6310
Epoch 8/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.7268 - loss: 0.6709 - val_accuracy: 0.7917 - val_loss

...

6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 664ms/step
Confusion Matrix:
[[56  0  0]
 [ 6 48  0]
 [ 0  6 52]]
Precision: 0.9307, Recall: 0.9285, Accuracy: 0.9286, F1 Score: 0.9278

=== Fold 5 ===
Class distribution: {2: 58, 0: 56, 1: 54}


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 91s 3s/step - accuracy: 0.3331 - loss: 1.1300 - val_accuracy: 0.3333 - val_loss: 1.0985
Epoch 2/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 135s 3s/step - accuracy: 0.3626 - loss: 1.0965 - val_accuracy: 0.3750 - val_loss: 1.0918
Epoch 3/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 79s 3s/step - accuracy: 0.3679 - loss: 1.0913 - val_accuracy: 0.3750 - val_loss: 1.0800
Epoch 4/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 87s 3s/step - accuracy: 0.4060 - loss: 1.0677 - val_accuracy: 0.5060 - val_loss: 1.0285
Epoch 5/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 138s 3s/step - accuracy: 0.5200 - loss: 0.9918 - val_accuracy: 0.5714 - val_loss: 0.9436
Epoch 6/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - accuracy: 0.6161 - loss: 0.8740 - val_accuracy: 0.7083 - val_loss: 0.7582
Epoch 7/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 86s 3s/step - accuracy: 0.7057 - loss: 0.6853 - val_accuracy: 0.7738 - val_loss: 0.5631
Epoch 8/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 136s 3s/step - accuracy: 0.8030 - loss: 0.4892 - val_accuracy: 0.8095 - val_lo

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 355s 3s/step - accuracy: 0.3907 - loss: 1.0770 - val_accuracy: 0.4714 - val_loss: 0.8672
Epoch 2/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 345s 3s/step - accuracy: 0.6861 - loss: 0.7036 - val_accuracy: 0.9048 - val_loss: 0.2550
Epoch 3/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 384s 3s/step - accuracy: 0.9252 - loss: 0.2253 - val_accuracy: 0.9381 - val_loss: 0.1750
Epoch 4/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 381s 3s/step - accuracy: 0.9698 - loss: 0.1114 - val_accuracy: 0.9857 - val_loss: 0.0546
Epoch 5/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 335s 3s/step - accuracy: 0.9622 - loss: 0.1272 - val_accuracy: 0.9929 - val_loss: 0.0331
Epoch 6/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 334s 3s/step - accuracy: 0.9906 - loss: 0.0251 - val_accuracy: 0.9929 - val_loss: 0.0401
Epoch 7/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 393s 3s/step - accuracy: 0.9981 - loss: 0.0116 - val_accuracy: 0.9929 - val_loss: 0.0223
Epoch 8/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 384s 3s/step - accuracy: 0.9974 - loss: 0.0102 - val_accu

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import TimeDistributed, Reshape, LSTM
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import os
from google.colab import drive

# Set random seed for reproducibility
tf.keras.utils.set_random_seed(492)


# Load data from Google Drive
def import_data(base_path, class_names, img_height=224, img_width=224):
    images = []
    labels = []
    for class_idx, class_name in enumerate(class_names):
        class_folder = os.path.join(base_path, class_name)
        for img_name in os.listdir(class_folder):
            img_path = os.path.join(class_folder, img_name)
            try:
                img = load_img(img_path, target_size=(img_height, img_width))
                img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
                images.append(img_array)
                labels.append(class_idx)
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
    return np.array(images), np.array(labels)

# Define CNN model

def create_cnn_lstm_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Reshape((54, 54*64)),
        LSTM(64, return_sequences=False),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])

    # Compile with Adam optimizer
    model.compile(optimizer=Adam(learning_rate=0.001),
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

    return model
# Main
base_path = '/content/drive/My Drive/mbpose/plank'
class_names = ['Incorrect_low', 'Correct', 'Incorrect_high']
img_height, img_width = 224, 224

# Load data
x, y = import_data(base_path, class_names, img_height, img_width)
num_classes = len(np.unique(y))

# Train-test split
x_train_full, x_test_full, y_train_full, y_test_full = train_test_split(
    x, y, test_size=0.2, stratify=y, random_state=42
)

# Stratified K-Fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_full), y=y_train_full)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

best_fold, best_f1 = None, -1
precision_list, recall_list, accuracy_list, f1_list = [], [], [], []

for fold_index, (train_index, val_index) in enumerate(skf.split(x_train_full, y_train_full), start=1):
    x_train, x_val = x_train_full[train_index], x_train_full[val_index]
    y_train, y_val = y_train_full[train_index], y_train_full[val_index]

    print(f"\nFold {fold_index} Class Distribution:")
    print(pd.Series(y_val).value_counts())

    # Create model
    model = create_cnn_lstm_model(input_shape=(img_height, img_width, 3), num_classes=num_classes)

    # Early stopping
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Train
    model.fit(
        x_train, y_train, epochs=30, batch_size=32, validation_data=(x_val, y_val),
        verbose=1, class_weight=class_weights, callbacks=[early_stop]
    )

    # Evaluate
    predictions = np.argmax(model.predict(x_val), axis=1)
    precision = precision_score(y_val, predictions, average='macro', zero_division=0)
    recall = recall_score(y_val, predictions, average='macro', zero_division=0)
    accuracy = accuracy_score(y_val, predictions)
    f1 = f1_score(y_val, predictions, average='macro', zero_division=0)

    # Confusion matrix
    print(f"Fold {fold_index} Confusion Matrix:")
    print(confusion_matrix(y_val, predictions))

    precision_list.append(precision)
    recall_list.append(recall)
    accuracy_list.append(accuracy)
    f1_list.append(f1)

    print(f"\n=== Fold {fold_index} ===")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")

    if f1 > best_f1:
        best_f1 = f1
        best_fold = fold_index

# Final metrics
print("\n=== Overall Metrics ===")
print(f"Avg Precision: {np.mean(precision_list):.4f}")
print(f"Avg Recall: {np.mean(recall_list):.4f}")
print(f"Avg Accuracy: {np.mean(accuracy_list):.4f}")
print(f"Avg F1 Score: {np.mean(f1_list):.4f}")
print(f"Best Fold: {best_fold} with F1 Score: {best_f1:.4f}")

# Test on held-out test set
test_predictions = np.argmax(model.predict(x_test_full), axis=1)
print("\n=== Test Set Metrics ===")
print(f"Test Accuracy: {accuracy_score(y_test_full, test_predictions):.4f}")
print(f"Test F1 Score: {f1_score(y_test_full, test_predictions, average='macro'):.4f}")


Fold 1 Class Distribution:
0    55
2    55
1    51
Name: count, dtype: int64


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 75s 3s/step - accuracy: 0.3486 - loss: 1.1477 - val_accuracy: 0.3416 - val_loss: 1.0985
Epoch 2/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 62s 3s/step - accuracy: 0.3101 - loss: 1.1096 - val_accuracy: 0.3416 - val_loss: 1.0952
Epoch 3/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - accuracy: 0.3228 - loss: 1.1114 - val_accuracy: 0.3727 - val_loss: 1.0850
Epoch 4/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.3644 - loss: 1.0913 - val_accuracy: 0.3789 - val_loss: 1.0641
Epoch 5/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 61s 3s/step - accuracy: 0.4320 - loss: 1.0202 - val_accuracy: 0.5714 - val_loss: 0.9340
Epoch 6/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 85s 3s/step - accuracy: 0.5493 - loss: 0.9140 - val_accuracy: 0.5776 - val_loss: 0.8395
Epoch 7/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.6712 - loss: 0.7228 - val_accuracy: 0.6398 - val_loss: 0.7944
Epoch 8/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 79s 3s/step - accuracy: 0.7493 - loss: 0.6293 - val_accuracy: 0.7205 - val_loss:

KeyboardInterrupt: 

(2) CNN-LSTM keypoints

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Set random seed for reproducibility
tf.keras.utils.set_random_seed(492)

# Load and return features and labels
def import_data():
    data = pd.read_csv('mergedshuffledPlankV2keypoints.csv')
    x = data.iloc[:, :-1].values
    y = data['correct'].astype(int).values
    y = np.searchsorted(np.unique(y), y)  # Make sure y is 0-based index
    return x, y

# Define the CNN-LSTM model
def create_cnn_lstm_model(input_shape, num_classes):
    model = Sequential([
        Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        LSTM(32, return_sequences=False),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Main script
x, y = import_data()
num_classes = len(np.unique(y))

# Create 6 stratified folds
skf = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)
folds = list(skf.split(x, y))
train_val_folds = folds[:5]
test_fold = folds[5]

precision_list, recall_list, accuracy_list, f1_list = [], [], [], []

# 5-Fold Cross-Validation
for fold_index, (train_index, val_index) in enumerate(train_val_folds, start=1):
    x_train_raw, x_val_raw = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Fit scaler only on training data
    scaler = MinMaxScaler()
    x_train = scaler.fit_transform(x_train_raw)
    x_val = scaler.transform(x_val_raw)

    # Reshape for CNN input
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
    x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))

    # Class weights for imbalance
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
    class_weights = {i: class_weights[i] for i in range(len(class_weights))}

    print(f"\n=== Fold {fold_index} ===")
    print("Class distribution:", pd.Series(y_val).value_counts().to_dict())

    # Train model
    model = create_cnn_lstm_model(input_shape=(x_train.shape[1], x_train.shape[2]), num_classes=num_classes)
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(x_train, y_train, epochs=30, batch_size=32,
              validation_data=(x_val, y_val),
              class_weight=class_weights,
              callbacks=[early_stop],
              verbose=1)

    # Evaluate
    preds = np.argmax(model.predict(x_val), axis=1)
    precision = precision_score(y_val, preds, average='macro', zero_division=0)
    recall = recall_score(y_val, preds, average='macro', zero_division=0)
    accuracy = accuracy_score(y_val, preds)
    f1 = f1_score(y_val, preds, average='macro', zero_division=0)

    print("Confusion Matrix:")
    print(confusion_matrix(y_val, preds))
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")

    precision_list.append(precision)
    recall_list.append(recall)
    accuracy_list.append(accuracy)
    f1_list.append(f1)

# Report average scores
print("\n=== 5-Fold Cross-Validation Summary ===")
print(f"Avg Precision: {np.mean(precision_list):.4f}")
print(f"Avg Recall:    {np.mean(recall_list):.4f}")
print(f"Avg Accuracy:  {np.mean(accuracy_list):.4f}")
print(f"Avg F1 Score:  {np.mean(f1_list):.4f}")

# === Final Testing on Fold 6 ===
train_index = np.concatenate([f[0] for f in train_val_folds])
test_index = test_fold[1]
x_train_raw, x_test_raw = x[train_index], x[test_index]
y_train, y_test = y[train_index], y[test_index]

# Refit scaler only on training data
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train_raw)
x_test = scaler.transform(x_test_raw)

# Reshape for CNN input
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

# Class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

# Train final model (Fold 6) with stricter early stopping
final_model = create_cnn_lstm_model(input_shape=(x_train.shape[1], x_train.shape[2]), num_classes=num_classes)
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=2,
    min_delta=0.001,
    restore_best_weights=True
)

final_model.fit(x_train, y_train, epochs=30, batch_size=32,
                validation_split=0.1, class_weight=class_weights,
                callbacks=[early_stop], verbose=1)

# Final test evaluation
test_preds = np.argmax(final_model.predict(x_test), axis=1)
test_accuracy = accuracy_score(y_test, test_preds)
test_f1 = f1_score(y_test, test_preds, average='macro')
test_precision = precision_score(y_test, test_preds, average='macro', zero_division=0)
test_recall = recall_score(y_test, test_preds, average='macro', zero_division=0)

print("\n=== Final Test Fold (Fold 6) Evaluation ===")
print(confusion_matrix(y_test, test_preds))
print(f"Test Accuracy:  {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall:    {test_recall:.4f}")
print(f"Test F1 Score:  {test_f1:.4f}")



=== Fold 1 ===
Class distribution: {2: 57, 0: 54, 1: 52}


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - accuracy: 0.3609 - loss: 1.0984 - val_accuracy: 0.3742 - val_loss: 1.0979
Epoch 2/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.3435 - loss: 1.0960 - val_accuracy: 0.3190 - val_loss: 1.0986
Epoch 3/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.3643 - loss: 1.0969 - val_accuracy: 0.3436 - val_loss: 1.0975
Epoch 4/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.3426 - loss: 1.0978 - val_accuracy: 0.3252 - val_loss: 1.0979
Epoch 5/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.3727 - loss: 1.0982 - val_accuracy: 0.3865 - val_loss: 1.0945
Epoch 6/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.3944 - loss: 1.0895 - val_accuracy: 0.4233 - val_loss: 1.0914
Epoch 7/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3804 - loss: 1.0881 - val_accuracy: 0.3620 - val_loss: 1.0849
Epoch 8/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4261 - loss: 1.0780 - val_accuracy: 0.4601 - v

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.3650 - loss: 1.1007 - val_accuracy: 0.3436 - val_loss: 1.0971
Epoch 2/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3169 - loss: 1.1081 - val_accuracy: 0.3681 - val_loss: 1.0967
Epoch 3/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.3285 - loss: 1.1057 - val_accuracy: 0.3681 - val_loss: 1.0954
Epoch 4/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3549 - loss: 1.1026 - val_accuracy: 0.3742 - val_loss: 1.0944
Epoch 5/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3408 - loss: 1.0996 - val_accuracy: 0.4110 - val_loss: 1.0927
Epoch 6/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3516 - loss: 1.0978 - val_accuracy: 0.3988 - val_loss: 1.0898
Epoch 7/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3503 - loss: 1.0965 - val_accuracy: 0.3988 - val_loss: 1.0855
Epoch 8/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.4233 - loss: 1.0918 - val_accuracy: 0.4540 - val_loss: 1.

...

26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - accuracy: 0.3084 - loss: 1.0993 - val_accuracy: 0.3642 - val_loss: 1.0973
Epoch 2/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.3519 - loss: 1.0971 - val_accuracy: 0.3333 - val_loss: 1.0982
Epoch 3/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.3359 - loss: 1.0981 - val_accuracy: 0.3148 - val_loss: 1.0984
Epoch 4/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.3199 - loss: 1.0973 - val_accuracy: 0.3580 - val_loss: 1.0973
Epoch 5/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.3586 - loss: 1.0974 - val_accuracy: 0.3148 - val_loss: 1.0966
Epoch 6/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3284 - loss: 1.0960 - val_accuracy: 0.3395 - val_loss: 1.0949
Epoch 7/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.3312 - loss: 1.0972 - val_accuracy: 0.4321 - val_loss: 1.0914
Epoch 8/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3905 - loss: 1.0907 - val_accuracy: 0.4691 - val_loss: 1.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.3403 - loss: 1.1007 - val_accuracy: 0.3148 - val_loss: 1.0995
Epoch 2/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3604 - loss: 1.1002 - val_accuracy: 0.3704 - val_loss: 1.0972
Epoch 3/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3366 - loss: 1.1026 - val_accuracy: 0.3580 - val_loss: 1.0971
Epoch 4/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3419 - loss: 1.1005 - val_accuracy: 0.3889 - val_loss: 1.0950
Epoch 5/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3528 - loss: 1.0999 - val_accuracy: 0.3704 - val_loss: 1.0933
Epoch 6/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3215 - loss: 1.1005 - val_accuracy: 0.4136 - val_loss: 1.0901
Epoch 7/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.3847 - loss: 1.0985 - val_accuracy: 0.4136 - val_loss: 1.0875
Epoch 8/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3688 - loss: 1.0966 - val_accuracy: 0.4136 - val_loss: 1.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


115/115 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.3427 - loss: 1.0995 - val_accuracy: 0.3719 - val_loss: 1.0946
Epoch 2/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.3737 - loss: 1.0940 - val_accuracy: 0.4064 - val_loss: 1.0871
Epoch 3/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.4163 - loss: 1.0805 - val_accuracy: 0.4655 - val_loss: 1.0512
Epoch 4/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.4941 - loss: 1.0141 - val_accuracy: 0.6650 - val_loss: 0.6993
Epoch 5/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7770 - loss: 0.5517 - val_accuracy: 0.8941 - val_loss: 0.2492
Epoch 6/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9005 - loss: 0.2777 - val_accuracy: 0.9680 - val_loss: 0.0868
Epoch 7/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9528 - loss: 0.1448 - val_accuracy: 0.9852 - val_loss: 0.0436
Epoch 8/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9765 - loss: 0.0770 - val_accuracy: 0.985

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import joblib

# Set random seed for reproducibility
tf.keras.utils.set_random_seed(492)

# Load and return features and labels
def import_data():
    data = pd.read_csv('mergedshuffledPlankV2keypoints.csv')
    x = data.iloc[:, :-1].values
    y = data['correct'].astype(int).values
    y = np.searchsorted(np.unique(y), y)  # Make sure y is 0-based index
    return x, y

# Define the CNN-LSTM model
def create_cnn_lstm_model(input_shape, num_classes):
    model = Sequential([
        Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        LSTM(32, return_sequences=False),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Main script
x, y = import_data()
num_classes = len(np.unique(y))

# Create 6 stratified folds
skf = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)
folds = list(skf.split(x, y))
train_val_folds = folds[:5]
test_fold = folds[5]

precision_list, recall_list, accuracy_list, f1_list = [], [], [], []

# Variables to track the best model
best_f1_score = -float('inf')
best_model = None
best_scaler = None
best_fold = 0

# 5-Fold Cross-Validation
for fold_index, (train_index, val_index) in enumerate(train_val_folds, start=1):
    x_train_raw, x_val_raw = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Fit scaler only on training data
    scaler = MinMaxScaler()
    x_train = scaler.fit_transform(x_train_raw)
    x_val = scaler.transform(x_val_raw)

    # Reshape for CNN input
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
    x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))

    # Class weights for imbalance
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
    class_weights = {i: class_weights[i] for i in range(len(class_weights))}

    print(f"\n=== Fold {fold_index} ===")
    print("Class distribution:", pd.Series(y_val).value_counts().to_dict())

    # Train model
    model = create_cnn_lstm_model(input_shape=(x_train.shape[1], x_train.shape[2]), num_classes=num_classes)
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(x_train, y_train, epochs=30, batch_size=32,
              validation_data=(x_val, y_val),
              class_weight=class_weights,
              callbacks=[early_stop],
              verbose=1)

    # Evaluate
    preds = np.argmax(model.predict(x_val), axis=1)
    precision = precision_score(y_val, preds, average='macro', zero_division=0)
    recall = recall_score(y_val, preds, average='macro', zero_division=0)
    accuracy = accuracy_score(y_val, preds)
    f1 = f1_score(y_val, preds, average='macro', zero_division=0)

    print("Confusion Matrix:")
    print(confusion_matrix(y_val, preds))
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")

    precision_list.append(precision)
    recall_list.append(recall)
    accuracy_list.append(accuracy)
    f1_list.append(f1)

    # Save the best model based on F1 score
    if f1 > best_f1_score:
        best_f1_score = f1
        best_model = model
        best_scaler = scaler
        best_fold = fold_index
        # Save the best model and scaler
        model.save('best_model.keras')
        joblib.dump(scaler, 'best_scaler.pkl')
        print(f"Saved best model and scaler from Fold {fold_index} with F1 Score: {f1:.4f}")

# Report average scores
print("\n=== 5-Fold Cross-Validation Summary ===")
print(f"Avg Precision: {np.mean(precision_list):.4f}")
print(f"Avg Recall:    {np.mean(recall_list):.4f}")
print(f"Avg Accuracy:  {np.mean(accuracy_list):.4f}")
print(f"Avg F1 Score:  {np.mean(f1_list):.4f}")

# === Final Testing on Fold 6 ===
train_index = np.concatenate([f[0] for f in train_val_folds])
test_index = test_fold[1]
x_train_raw, x_test_raw = x[train_index], x[test_index]
y_train, y_test = y[train_index], y[test_index]

# Refit scaler only on training data
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train_raw)
x_test = scaler.transform(x_test_raw)

# Reshape for CNN input
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

# Class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

# Train final model (Fold 6) with stricter early stopping
final_model = create_cnn_lstm_model(input_shape=(x_train.shape[1], x_train.shape[2]), num_classes=num_classes)
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=2,
    min_delta=0.001,
    restore_best_weights=True
)

final_model.fit(x_train, y_train, epochs=30, batch_size=32,
                validation_split=0.1, class_weight=class_weights,
                callbacks=[early_stop], verbose=1)

# Final test evaluation
test_preds = np.argmax(final_model.predict(x_test), axis=1)
test_accuracy = accuracy_score(y_test, test_preds)
test_f1 = f1_score(y_test, test_preds, average='macro')
test_precision = precision_score(y_test, test_preds, average='macro', zero_division=0)
test_recall = recall_score(y_test, test_preds, average='macro', zero_division=0)

print("\n=== Final Test Fold (Fold 6) Evaluation ===")
print(confusion_matrix(y_test, test_preds))
print(f"Test Accuracy:  {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall:    {test_recall:.4f}")
print(f"Test F1 Score:  {test_f1:.4f}")

print(f"\n=== Best Model Summary ===")
print(f"Best model is from Fold {best_fold} with F1 Score: {best_f1_score:.4f}")


=== Fold 1 ===
Class distribution: {2: 57, 0: 54, 1: 52}


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.3609 - loss: 1.0984 - val_accuracy: 0.3742 - val_loss: 1.0979
Epoch 2/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3435 - loss: 1.0960 - val_accuracy: 0.3190 - val_loss: 1.0986
Epoch 3/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3643 - loss: 1.0969 - val_accuracy: 0.3436 - val_loss: 1.0975
Epoch 4/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3426 - loss: 1.0978 - val_accuracy: 0.3252 - val_loss: 1.0979
Epoch 5/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3727 - loss: 1.0982 - val_accuracy: 0.3865 - val_loss: 1.0945
Epoch 6/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3944 - loss: 1.0895 - val_accuracy: 0.4233 - val_loss: 1.0914
Epoch 7/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3804 - loss: 1.0881 - val_accuracy: 0.3620 - val_loss: 1.0849
Epoch 8/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4261 - loss: 1.0780 - val_accuracy: 0.4601 - val_

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.3650 - loss: 1.1007 - val_accuracy: 0.3436 - val_loss: 1.0971
Epoch 2/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3169 - loss: 1.1081 - val_accuracy: 0.3681 - val_loss: 1.0967
Epoch 3/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3285 - loss: 1.1057 - val_accuracy: 0.3681 - val_loss: 1.0954
Epoch 4/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3549 - loss: 1.1026 - val_accuracy: 0.3742 - val_loss: 1.0944
Epoch 5/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3408 - loss: 1.0996 - val_accuracy: 0.4110 - val_loss: 1.0927
Epoch 6/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3516 - loss: 1.0978 - val_accuracy: 0.3988 - val_loss: 1.0898
Epoch 7/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3503 - loss: 1.0965 - val_accuracy: 0.3988 - val_loss: 1.0855
Epoch 8/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4233 - loss: 1.0918 - val_accuracy: 0.4540 - val_loss: 1.0757
E

...

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Confusion Matrix:
[[52  1  0]
 [ 0 50  1]
 [ 0  8 50]]
Precision: 0.9426, Recall: 0.9412, Accuracy: 0.9383, F1 Score: 0.9390

=== Fold 5 ===
Class distribution: {2: 58, 0: 53, 1: 51}
Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.3403 - loss: 1.1007 - val_accuracy: 0.3148 - val_loss: 1.0995
Epoch 2/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3604 - loss: 1.1002 - val_accuracy: 0.3704 - val_loss: 1.0972
Epoch 3/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.3366 - loss: 1.1026 - val_accuracy: 0.3580 - val_loss: 1.0971
Epoch 4/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3419 - loss: 1.1005 - val_accuracy: 0.3889 - val_loss: 1.0950
Epoch 5/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3528 - loss: 1.0999 - val_accuracy: 0.3704 - val_loss: 1.0933
Epoch 6/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3215 - loss: 1.1005 - val_accuracy: 0.4136 - val_loss: 1.0901
Epoch 7/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3847 - loss: 1.0985 - val_accuracy: 0.4136 - val_loss: 1.0875
Epoch 8/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3688 - loss: 1.0966 - val_accuracy: 0.4136 - val_loss: 1.0806

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


115/115 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.3427 - loss: 1.0995 - val_accuracy: 0.3719 - val_loss: 1.0946
Epoch 2/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.3737 - loss: 1.0940 - val_accuracy: 0.4064 - val_loss: 1.0871
Epoch 3/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4163 - loss: 1.0805 - val_accuracy: 0.4655 - val_loss: 1.0512
Epoch 4/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4941 - loss: 1.0141 - val_accuracy: 0.6650 - val_loss: 0.6993
Epoch 5/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7770 - loss: 0.5517 - val_accuracy: 0.8941 - val_loss: 0.2492
Epoch 6/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9005 - loss: 0.2777 - val_accuracy: 0.9680 - val_loss: 0.0868
Epoch 7/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9528 - loss: 0.1448 - val_accuracy: 0.9852 - val_loss: 0.0436
Epoch 8/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9765 - loss: 0.0770 - val_accuracy: 0.9852 - va

## (3) CNN-LSTM augmentation data

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Set random seed for reproducibility
tf.keras.utils.set_random_seed(492)

# Load data
def import_data():
    data = pd.read_csv('mergedshuffledPlankV2keypoints.csv')
    x = data.iloc[:, :-1].values
    y = data['correct'].astype(int).values
    y = np.searchsorted(np.unique(y), y)
    scaler = MinMaxScaler()
    x = scaler.fit_transform(x)
    return x, y

# Balance dataset using SMOTE
def balance_data(x, y):
    smote = SMOTE(sampling_strategy='auto', random_state=42)
    x_resampled, y_resampled = smote.fit_resample(x, y)
    print("Balanced class distribution:")
    print(pd.Series(y_resampled).value_counts())
    return x_resampled, y_resampled

# Add Gaussian noise
def augment_data(x, noise_factor=0.05):
    noise = np.random.normal(loc=0.0, scale=noise_factor, size=x.shape)
    return x + noise

# Model definition
def create_cnn_lstm_model(input_shape, num_classes):
    model = Sequential([
        Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        LSTM(64, return_sequences=False),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Main
x, y = import_data()
x, y = balance_data(x, y)
num_classes = len(np.unique(y))

# Create 6 stratified folds
skf = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)
folds = list(skf.split(x, y))
train_val_folds = folds[:5]
test_fold = folds[5]

# Initialise trackers
best_model_path = "best_cnn_lstm_model.keras"
best_fold, best_f1 = None, -1
precision_list, recall_list, accuracy_list, f1_list = [], [], [], []

# Perform 5-Fold CV on first 5 folds
for fold_index, (train_index, val_index) in enumerate(train_val_folds, start=1):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Augment training data
    x_train = augment_data(x_train, noise_factor=0.05)

    # Reshape
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
    x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))

    # Class weights
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
    class_weights = {i: class_weights[i] for i in range(len(class_weights))}

    print(f"\n=== Fold {fold_index} ===")
    print("Class distribution:", pd.Series(y_val).value_counts().to_dict())

    # Model & Callbacks
    model = create_cnn_lstm_model(input_shape=(x_train.shape[1], x_train.shape[2]), num_classes=num_classes)
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(
        x_train, y_train,
        epochs=30,
        batch_size=32,
        validation_data=(x_val, y_val),
        class_weight=class_weights,
        callbacks=[early_stop],
        verbose=1
    )

    # Evaluate
    preds = np.argmax(model.predict(x_val), axis=1)
    precision = precision_score(y_val, preds, average='macro', zero_division=0)
    recall = recall_score(y_val, preds, average='macro', zero_division=0)
    accuracy = accuracy_score(y_val, preds)
    f1 = f1_score(y_val, preds, average='macro', zero_division=0)

    print("Confusion Matrix:")
    print(confusion_matrix(y_val, preds))
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")

    # Save metrics
    precision_list.append(precision)
    recall_list.append(recall)
    accuracy_list.append(accuracy)
    f1_list.append(f1)

    if f1 > best_f1:
        best_f1 = f1
        best_fold = fold_index
        model.save(best_model_path)

# Print summary
print("\n=== 5-Fold Validation Summary ===")
print(f"Avg Precision: {np.mean(precision_list):.4f}")
print(f"Avg Recall:    {np.mean(recall_list):.4f}")
print(f"Avg Accuracy:  {np.mean(accuracy_list):.4f}")
print(f"Avg F1 Score:  {np.mean(f1_list):.4f}")
print(f"Best Fold: {best_fold} with F1 Score: {best_f1:.4f}")
print(f"Best model saved at: {best_model_path}")

# === Final Test Fold Evaluation ===
test_index = test_fold[1]
train_index = np.concatenate([f[0] for f in train_val_folds])
x_train, x_test = x[train_index], x[test_index]
y_train, y_test = y[train_index], y[test_index]

# Augment and reshape
x_train = augment_data(x_train, noise_factor=0.05)
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

# Class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

# Final model training on Fold 6
final_model = create_cnn_lstm_model(input_shape=(x_train.shape[1], x_train.shape[2]), num_classes=num_classes)
early_stop = EarlyStopping(monitor='val_accuracy', patience=2, min_delta=0.001, restore_best_weights=True)

final_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=30,
    batch_size=32,
    class_weight=class_weights,
    callbacks=[early_stop],
    verbose=1
)

# Evaluate on final test set
test_preds = np.argmax(final_model.predict(x_test), axis=1)
test_accuracy = accuracy_score(y_test, test_preds)
test_f1 = f1_score(y_test, test_preds, average='macro')
test_precision = precision_score(y_test, test_preds, average='macro', zero_division=0)
test_recall = recall_score(y_test, test_preds, average='macro', zero_division=0)

print("\n=== Final Test Fold (Fold 6) Evaluation ===")
print(confusion_matrix(y_test, test_preds))
print(f"Test Accuracy:  {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall:    {test_recall:.4f}")
print(f"Test F1 Score:  {test_f1:.4f}")

Balanced class distribution:
1    346
2    346
0    346
Name: count, dtype: int64

=== Fold 1 ===
Class distribution: {1: 58, 2: 58, 0: 57}


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 17s 77ms/step - accuracy: 0.3456 - loss: 1.0981 - val_accuracy: 0.3931 - val_loss: 1.0960
Epoch 2/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.3325 - loss: 1.0990 - val_accuracy: 0.3353 - val_loss: 1.0945
Epoch 3/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3364 - loss: 1.0985 - val_accuracy: 0.3353 - val_loss: 1.0915
Epoch 4/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3585 - loss: 1.0968 - val_accuracy: 0.4104 - val_loss: 1.0820
Epoch 5/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3956 - loss: 1.0844 - val_accuracy: 0.3815 - val_loss: 1.0764
Epoch 6/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4046 - loss: 1.0873 - val_accuracy: 0.4740 - val_loss: 1.0533
Epoch 7/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4085 - loss: 1.0506 - val_accuracy: 0.4798 - val_loss: 1.0268
Epoch 8/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.4894 - loss: 1.0179 - val_accuracy: 0.6705 - 

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - accuracy: 0.3397 - loss: 1.1003 - val_accuracy: 0.3353 - val_loss: 1.0985
Epoch 2/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3239 - loss: 1.1022 - val_accuracy: 0.3584 - val_loss: 1.0975
Epoch 3/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3598 - loss: 1.0975 - val_accuracy: 0.3353 - val_loss: 1.0980
Epoch 4/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.3567 - loss: 1.0973 - val_accuracy: 0.3353 - val_loss: 1.0967
Epoch 5/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.3339 - loss: 1.0964 - val_accuracy: 0.3353 - val_loss: 1.0952
Epoch 6/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3632 - loss: 1.0937 - val_accuracy: 0.3873 - val_loss: 1.0835
Epoch 7/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3796 - loss: 1.0883 - val_accuracy: 0.4740 - val_loss: 1.0715
Epoch 8/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4209 - loss: 1.0771 - val_accuracy: 0.4682 - val_loss: 1.

...

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Confusion Matrix:
[[55  3  0]
 [ 0 53  5]
 [ 0  1 56]]
Precision: 0.9493, Recall: 0.9482, Accuracy: 0.9480, F1 Score: 0.9481

=== Fold 5 ===
Class distribution: {2: 58, 0: 58, 1: 57}
Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 0.3096 - loss: 1.1005 - val_accuracy: 0.3295 - val_loss: 1.0989
Epoch 2/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.3132 - loss: 1.1002 - val_accuracy: 0.3295 - val_loss: 1.0985
Epoch 3/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.3255 - loss: 1.1014 - val_accuracy: 0.3353 - val_loss: 1.0955
Epoch 4/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3345 - loss: 1.0988 - val_accuracy: 0.3295 - val_loss: 1.0951
Epoch 5/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3104 - loss: 1.1009 - val_accuracy: 0.3873 - val_loss: 1.0861
Epoch 6/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3677 - loss: 1.0905 - val_accuracy: 0.3873 - val_loss: 1.0711
Epoch 7/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3907 - loss: 1.0872 - val_accuracy: 0.3815 - val_loss: 1.0862
Epoch 8/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.3621 - loss: 1.0921 - val_accuracy: 0.4682 - val_loss: 1.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


122/122 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.3430 - loss: 1.0996 - val_accuracy: 0.3741 - val_loss: 1.0902
Epoch 2/30
122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.3615 - loss: 1.0900 - val_accuracy: 0.5427 - val_loss: 0.9641
Epoch 3/30
122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.6251 - loss: 0.8091 - val_accuracy: 0.8129 - val_loss: 0.4200
Epoch 4/30
122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8529 - loss: 0.3781 - val_accuracy: 0.9215 - val_loss: 0.2477
Epoch 5/30
122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9110 - loss: 0.2559 - val_accuracy: 0.9376 - val_loss: 0.1704
Epoch 6/30
122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9307 - loss: 0.1926 - val_accuracy: 0.9376 - val_loss: 0.1724
Epoch 7/30
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9382 - loss: 0.1722 - val_accuracy: 0.9330 - val_loss: 0.1641
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step

=== Final Test Fold (Fold 6) Evaluation ===
[[58  0  0]
 [ 0 57  0]
 